# Clustering Crypto

In [207]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json

### Fetching Cryptocurrency Data

In [213]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

In [214]:
# Create a DataFrame 
crypto_df = pd.DataFrame(json_data['Data']).T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,BlockTime,AssetLaunchDate,MaxSupply,MktCapPenalty,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,IsUsedInDefi
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,0,2017-07-01,300,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame

### Data Preprocessing

In [217]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

new_crypto_df = crypto_df.drop(columns=['Id','Url','ImageUrl','ContentCreatedOn','Name','Symbol','FullName','Description', 'AssetTokenStatus','BlockTime','AssetLaunchDate', 'MktCapPenalty', 'PlatformType', 'BuiltOn', 'SmartContractAddress', 'DecimalPoints', 'Difficulty', 'Taxonomy', 'Sponsored', 'SortOrder', 'Rating','NetHashesPerSecond','BlockNumber','BlockReward','IsUsedInDefi' ])


In [218]:
# Keep only cryptocurrencies that are trading
is_trading_df = new_crypto_df[new_crypto_df.IsTrading == True]
is_trading_df.head(10)

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,0,0
300,300 token,N/A,N/A,True,300,300
365,365Coin,X11,PoW/PoS,True,0,0
404,404Coin,Scrypt,PoW/PoS,True,0,0
611,SixEleven,SHA-256,PoW,True,0,0
808,808,SHA-256,PoW/PoS,True,0,0
888,Octocoin,N/A,PoW,True,0,0
1337,EliteCoin,X13,PoW/PoS,True,0,0
2015,2015 coin,X11,PoW/PoS,True,0,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,True,NaN,NaN


In [219]:
# Keep only cryptocurrencies with a working algorithm
working_algo_df = is_trading_df.dropna(axis=0, subset=["Algorithm"])
working_algo_df.head(10)

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,0,0
300,300 token,N/A,N/A,True,300,300
365,365Coin,X11,PoW/PoS,True,0,0
404,404Coin,Scrypt,PoW/PoS,True,0,0
611,SixEleven,SHA-256,PoW,True,0,0
808,808,SHA-256,PoW/PoS,True,0,0
888,Octocoin,N/A,PoW,True,0,0
1337,EliteCoin,X13,PoW/PoS,True,0,0
2015,2015 coin,X11,PoW/PoS,True,0,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,True,NaN,NaN


In [220]:
# Remove the "IsTrading" column
newer_crypto_df = working_algo_df.drop("IsTrading", axis=1)
newer_crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
300,300 token,N/A,N/A,300,300
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
888,Octocoin,N/A,PoW,0,0
1337,EliteCoin,X13,PoW/PoS,0,0
2015,2015 coin,X11,PoW/PoS,0,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,NaN,NaN


In [221]:
# Remove rows with at least 1 null value
newer_crypto_df.dropna()
newer_crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
300,300 token,N/A,N/A,300,300
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
888,Octocoin,N/A,PoW,0,0
1337,EliteCoin,X13,PoW/PoS,0,0
2015,2015 coin,X11,PoW/PoS,0,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,NaN,NaN


In [222]:
# Remove rows with cryptocurrencies having no coins mined
newer_crypto_df.dropna(inplace=True)
newer_crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
300,300 token,N/A,N/A,300,300
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
888,Octocoin,N/A,PoW,0,0
1337,EliteCoin,X13,PoW/PoS,0,0
2015,2015 coin,X11,PoW/PoS,0,0
GEO,GeoCoin,N/A,N/A,0,0


In [223]:
# Drop rows where there are 'N/A' text values
newest_crypto_df = newer_crypto_df.dropna(axis=1)

In [224]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df

coin_name_df = pd.DataFrame(newest_crypto_df["CoinName"])
coin_name_df.head()




,CoinName
42,42 Coin
300,300 token
365,365Coin
404,404Coin
611,SixEleven


In [225]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
updated_df = newest_crypto_df.drop(["CoinName"],axis=1)
updated_df.head

<bound method NDFrame.head of       Algorithm ProofType TotalCoinsMined    MaxSupply
42       Scrypt   PoW/PoS               0            0
300         N/A       N/A             300          300
365         X11   PoW/PoS               0            0
404      Scrypt   PoW/PoS               0            0
611     SHA-256       PoW               0            0
...         ...       ...             ...          ...
NEO     dBFT2.0       N/A       100000000    100000000
XRP         N/A       N/A     99990820713  99990820713
EOS        DPoS      DPoS     1.02564e+09           -1
ESD         N/A       N/A     5.10924e+08  5.10924e+08
PYLON       N/A       N/A           14700        14700

[1024 rows x 4 columns]>

In [226]:
# Create dummy variables for text features
dummy_vars = pd.get_dummies(updated_df, columns=["Algorithm", "ProofType"])
dummy_vars.head(10)

,TotalCoinsMined,MaxSupply,Algorithm_Argon2d,Algorithm_Autolykos,Algorithm_BEP-2,Algorithm_BEP2 Token,Algorithm_BEP20 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,...,ProofType_SCP,ProofType_SPoS,ProofType_TPoS,ProofType_Tangle,ProofType_Zero-Knowledge Proof,ProofType_dPoS,ProofType_dPoS/BFT,ProofType_dPoW/PoW,ProofType_ePoW,ProofType_fPoW+ PBFT
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
300,300,300,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GEO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [227]:
# Standardize data
X_scaled = StandardScaler().fit_transform(dummy_vars)
print(X_scaled[0:5])

[[-0.03290203 -0.03287921 -0.03126527 -0.03126527 -0.03126527 -0.0442374
  -0.03126527 -0.0442374  -0.03126527 -0.03126527 -0.0442374  -0.0442374
  -0.0767718  -0.0442374  -0.03126527 -0.03126527 -0.08873565 -0.03126527
  -0.03126527 -0.03126527 -0.03126527 -0.08873565 -0.03126527 -0.03126527
  -0.08873565 -0.0442374  -0.03126527 -0.0442374  -0.03126527 -0.03126527
  -0.03126527 -0.03126527 -0.0442374  -0.03126527 -0.03126527 -0.06262243
  -2.14374169 -0.05420605 -0.05420605 -0.03126527 -0.0442374  -0.08873565
  -0.03126527 -0.03126527 -0.0767718  -0.03126527 -0.15491933 -0.0442374
  -0.03126527 -0.03126527  5.75615612 -0.03126527 -0.03126527 -0.03126527
  -0.03126527 -0.03126527 -0.03126527 -0.03126527 -0.08873565 -0.03126527
  -0.09416472 -0.03126527 -0.03126527 -0.03126527 -0.03126527 -0.03126527
  -0.11773439 -0.03126527 -0.03126527 -0.03126527 -1.80739223 -0.06262243
  -0.0442374  -0.03126527 -0.03126527 -0.03126527 -0.03126527 -0.22894368
  -0.03126527 -0.0442374  -0.03126527 -0.

### Reducing Dimensions Using PCA

In [228]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[ 4.20435738, -0.16719896, -0.39918542],
       [-0.95068413, -0.01515376, -0.00755087],
       [ 4.14110145, -0.30775634, -0.13786292],
       ...,
       [ 2.99204989,  0.0333473 , -0.14206808],
       [-0.95068577, -0.01514232, -0.00754191],
       [-0.95068413, -0.01515376, -0.00755087]])

In [229]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=X_pca,
    columns=['PC 1', 'PC 2', 'PC 3'],
    index=dummy_vars.index
)
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,4.204357,-0.167199,-0.399185
300,-0.950684,-0.015154,-0.007551
365,4.141101,-0.307756,-0.137863
404,4.204357,-0.167199,-0.399185
611,3.977527,-0.146166,-0.350348
808,4.086727,-0.142046,-0.367541
888,1.432479,-0.100801,-0.185524
1337,4.080582,-0.165833,-0.354351
2015,4.141101,-0.307756,-0.137863
GEO,-0.950684,-0.015154,-0.007551


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [230]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [234]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
predictions
# Create a new DataFrame including predicted clusters and cryptocurrencies features
cryptocluster_df = pd.concat([updated_df, pcs_df], axis=1)
cryptocluster_df["Class"] = model.labels_
cryptocluster_df['CoinName'] = coin_name_df["CoinName"]

### Visualizing Results

#### 3D-Scatter with Clusters

In [236]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    cryptocluster_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=1000,
    hover_name = "CoinName",
    hover_data = ["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


#### Table of Tradable Cryptocurrencies

In [237]:
# Table with tradable cryptos
cryptocluster_df.hvplot.table()

:Table   [Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC 1,PC 2,PC 3,Class,CoinName]

In [238]:
# Print the total number of tradable cryptocurrencies
print(f' There are {cryptocluster_df["MaxSupply"].count()} tradable cryptocurrencies')

 There are 1024 tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [241]:
# Scale data to create the scatter plot
scaling_data = cryptocluster_df.copy()
scaling_data_2 = scaling_data.drop(
    ["Algorithm", "ProofType","PC 1", "PC 2", "PC 3", "CoinName" , "Class"], 
    axis=1
)
min_max = MinMaxScaler().fit_transform(scaling_data_2)
min_max

array([[0.00000000e+00, 2.01668509e-15],
       [6.05005527e-13, 6.07022212e-13],
       [0.00000000e+00, 2.01668509e-15],
       ...,
       [2.06838937e-06, 0.00000000e+00],
       [1.03037298e-06, 1.03037298e-06],
       [2.96452708e-11, 2.96472875e-11]])

In [244]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
min_max_df = pd.DataFrame(
    data=min_max,
    columns=["TotalCoinsMined", "MaxSupply"],
    index=clustered_df.index
)

min_max_df.head

# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
min_max_df["CoinName"] = coin_name_df["CoinName"]
min_max_df["Class"] = cryptocluster_df["Class"] 
min_max_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="MaxSupply",
    by="Class",
    hover_cols=["CoinName"],
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (MaxSupply,CoinName)